In [ ]:
from os.path import exists

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!mkdir anon

In [ ]:
import shutil 
shutil.unpack_archive("drive/MyDrive/wea.zip", "anon/")

In [ ]:
import os
import torch
from torch import nn
#from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [ ]:
# Configuration options
k_folds = 3
num_epochs = 1
loss_function = nn.CrossEntropyLoss()
  
  # For fold results
results = {}
  
  # Set fixed random number seed
torch.manual_seed(42)
  

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
data_dir  = '/content/anon/wea'

In [ ]:
train_dataset = datasets.ImageFolder(root='/content/anon/wea/train', transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root='/content/anon/wea/val', transform=data_transforms['val'])
dataset = ConcatDataset([train_dataset, test_dataset])

In [ ]:
kfold = KFold(n_splits=k_folds, shuffle=True)

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


In [ ]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
  # Print
  print(f'FOLD {fold}')
  print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
  trainloader = torch.utils.data.DataLoader(
                     dataset, 
                      batch_size=10, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network
  network = model_ft 
  #network.apply(reset_weights)
  
    # Initialize optimizer
  optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    
    # Run the training loop for defined number of epochs
  for epoch in range(0, 10):

      # Print epoch
    print(f'Starting epoch {epoch+1}')

      # Set current loss value
    current_loss = 0.0

      # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
        
        # Get inputs
      inputs, targets = data
      
        # Zero the gradients
      optimizer.zero_grad()
        
        # Perform forward pass
      outputs = network(inputs)
        
        # Compute loss
      loss = loss_function(outputs, targets)
        
        # Perform backward pass
      loss.backward()
        
        # Perform optimization
      optimizer.step()
      exp_lr_scheduler .step()
        # Print statistics
      current_loss += loss.item()
      if i % 100 == 99:
        print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 100))
        current_loss = 0.0
            
    # Process is complete.
  print('Training process has finished. Saving trained model.')

    # Print about testing
  print('Starting testing')
    
    # Saving the model
  save_path = f'./model-fold-{fold}.pth'
  torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
  correct, total = 0, 0
  with torch.no_grad():

      # Iterate over the test data and generate predictions
    for i, data in enumerate(testloader, 0):

        # Get inputs
      inputs, targets = data

        # Generate outputs
      outputs = network(inputs)

        # Set total and correct
      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

      # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)
    
  # Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

FOLD 0
--------------------------------
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Training process has finished. Saving trained model.
Starting testing
Accuracy for fold 0: 90 %
--------------------------------
FOLD 1
--------------------------------
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Training process has finished. Saving trained model.
Starting testing
Accuracy for fold 1: 91 %
--------------------------------
FOLD 2
--------------------------------
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Training process has finished. Saving trained model.
Starting testing
Accuracy for fold 2: 94 %
-------------